In [38]:
import pandas as pd 
df1=pd.read_csv('sejong_fire.csv',encoding='cp949')
end=df1['상황종료일시']
start=df1['접수일시']
df1['진압시간']=(pd.to_datetime(df1['상황종료일시'])-pd.to_datetime(df1['접수일시']))
name=df1['진압시간'].sort_values(ascending=False).index[0]
mm=df1[df1['관할서명']==df1.loc[name,'관할서명']]['진압시간']
mm.astype('timedelta64[s]').mean()

Timedelta('0 days 10:20:43')

In [43]:
df2=pd.read_csv('busan_school_Info.csv',encoding='cp949')
df21=df2.copy()
df21.drop(columns=['school_name','teacher'],inplace=True)
df2['교사비율']=df21.sum(axis=1)/df2.teacher
max_idx=df2['교사비율'].argmax()
print(df2.loc[max_idx,'teacher'])

24


In [111]:
df3=pd.read_csv('five_crime.csv',encoding='cp949')
df3['범죄발생건수']=df3[df3.columns[range(1,11,2)]].sum(axis=1)
df3['연']=df3['연월'].str.replace(' ','').str[:4]
max_idx=df3.groupby(df3['연'])['범죄발생건수'].mean().idxmax()
df3[df3['연']==max_idx]['폭력_검거건수'].mean()

4884.166666666667

In [139]:
X_train=pd.read_csv('Airline_train.csv',encoding='cp949')
X_test=pd.read_csv('Airline_test.csv',encoding='cp949')
#X_train.info()

y_train=X_train.Satisfaction
y_train.replace({'Yes':'Satisfied','No':'Not satisfied'},inplace=True)
X_train.drop(columns='Satisfaction',inplace=True)

obj=X_train.columns[:4]
nums=X_train.columns[4:]

from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,stratify=y_train,random_state=69,test_size=0.2)

from sklearn.preprocessing import OneHotEncoder,StandardScaler
Encoder=OneHotEncoder(sparse=False)
Encoder.fit(X_train[obj])
X_train_oh=Encoder.transform(X_train[obj])
X_val_oh=Encoder.transform(X_val[obj])
X_test_oh=Encoder.transform(X_test[obj])

sclaer=StandardScaler()
sclaer.fit(X_train[nums])
X_train_sc=sclaer.transform(X_train[nums])
X_val_sc=sclaer.transform(X_val[nums])
X_test_sc=sclaer.transform(X_test[nums])

import numpy as np

X_Train=np.concatenate([X_train_oh,X_train_sc],axis=1)
X_Val=np.concatenate([X_val_oh,X_val_sc],axis=1)
X_Test=np.concatenate([X_test_oh,X_test_sc],axis=1)

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import f1_score

# rf
rf_model=RandomForestClassifier(n_estimators=200)
rf_model.fit(X_Train,y_train)
rf_pred=rf_model.predict(X_Val)
rf_score=f1_score(y_val,rf_pred,average='macro')
print('rf',rf_score)

# # ada
# ada_model=AdaBoostClassifier()
# ada_model.fit(X_Train,y_train)
# ada_pred=ada_model.predict(X_Val)
# ada_score=f1_score(y_val,ada_pred,average='macro')
# print('ada',ada_score)

pred=rf_model.predict(X_Test)
result=pd.DataFrame({'pred':pred})
result.to_csv('result.csv',index=False)


c:\Users\KDP-30\anaconda3\envs\ML_38\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


rf 0.9522914181433233


In [143]:
df31=pd.read_csv('cancer.csv')
df31.isna().sum()

x    81
dtype: int64